In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score

# Load dataset
df = pd.read_csv("dataset_uji_fuzzy.csv")

# Fungsi keanggotaan linear naik dan turun
def turun(a, b, x):
    return np.clip((a - x) / (a - b), 0, 1)

def naik(a, b, x):
    return np.clip((x - a) / (b - a), 0, 1)

def agregasi_turun(a, b, alpha):
    return a - alpha * (a - b)

def agregasi_naik(a, b, alpha):
    return alpha * (b - a) + a

# Kategori fuzzy untuk output
kategori_output = {
    'dingin': (17, 15),
    'cukup_dingin': (19, 17),
    'normal': (21, 19),
    'cukup_hangat': (23, 21),
    'hangat': (25, 23)
}

# Rule base
aturan = [
    ('sejuk', 'dingin', 'sedikit', 'cukup_dingin'),
    ('sejuk', 'dingin', 'sedang', 'normal'),
    ('sejuk', 'dingin', 'banyak', 'cukup_hangat'),
    ('sejuk', 'normal', 'sedikit', 'normal'),
    ('sejuk', 'normal', 'sedang', 'cukup_hangat'),
    ('sejuk', 'normal', 'banyak', 'hangat'),
    ('sejuk', 'hangat', 'sedikit', 'cukup_hangat'),
    ('sejuk', 'hangat', 'sedang', 'hangat'),
    ('sejuk', 'hangat', 'banyak', 'hangat'),
    ('normal', 'dingin', 'sedikit', 'normal'),
    ('normal', 'dingin', 'sedang', 'cukup_hangat'),
    ('normal', 'dingin', 'banyak', 'hangat'),
    ('normal', 'normal', 'sedikit', 'cukup_hangat'),
    ('normal', 'normal', 'sedang', 'hangat'),
    ('normal', 'normal', 'banyak', 'hangat'),
    ('normal', 'hangat', 'sedikit', 'hangat'),
    ('normal', 'hangat', 'sedang', 'hangat'),
    ('normal', 'hangat', 'banyak', 'hangat'),
    ('panas', 'dingin', 'sedikit', 'cukup_dingin'),
    ('panas', 'dingin', 'sedang', 'dingin'),
    ('panas', 'dingin', 'banyak', 'dingin'),
    ('panas', 'normal', 'sedikit', 'cukup_dingin'),
    ('panas', 'normal', 'sedang', 'dingin'),
    ('panas', 'normal', 'banyak', 'dingin'),
    ('panas', 'hangat', 'sedikit', 'dingin'),
    ('panas', 'hangat', 'sedang', 'dingin'),
    ('panas', 'hangat', 'banyak', 'dingin'),
]

# Fuzzifikasi 
def fuzzify(sl, sd, jo):
    return {
        'suhu_luar': {
            'sejuk': turun(24, 10, sl),
            'normal': min(naik(22, 24, sl), turun(33, 22, sl)),
            'panas': naik(32, 40, sl)
        },
        'suhu_dalam': {
            'dingin': turun(24, 10, sd),
            'normal': min(naik(22, 24, sd), turun(31, 22, sd)),
            'hangat': naik(30, 40, sd)
        },
        'jumlah_orang': {
            'sedikit': turun(18, 0, jo),
            'sedang': min(naik(12, 34, jo), turun(34, 12, jo)),
            'banyak': naik(30, 50, jo)
        }
    }

# Inferensi Tsukamoto
def tsukamoto(sl, sd, jo):
    fuzzy = fuzzify(sl, sd, jo)
    z_list, alpha_list = [], []

    for rule in aturan:
        a, b, c, output = rule
        alpha = min(fuzzy['suhu_luar'][a], fuzzy['suhu_dalam'][b], fuzzy['jumlah_orang'][c])
        atas, bawah = kategori_output[output]
        z = agregasi_turun(atas, bawah, alpha) if output in ['dingin', 'cukup_dingin'] else agregasi_naik(bawah, atas, alpha)
        z_list.append(z)
        alpha_list.append(alpha)

    return np.sum(np.multiply(alpha_list, z_list)) / np.sum(alpha_list) if np.sum(alpha_list) != 0 else 0

# Proses prediksi seluruh dataset
df['prediksi'] = df.apply(lambda row: round(tsukamoto(row['suhu_luar'], row['suhu_dalam'], row['jumlah_orang']), 1), axis=1)

# Deteksi kolom aktual
if 'suhu_optimal_aktual' in df.columns:
    df['suhu_aktual'] = df['suhu_optimal_aktual']
else:
    alt_cols = ['target', 'actual', 'suhu_target', 'actual_temp']
    for col in alt_cols:
        if col in df.columns:
            df['suhu_aktual'] = df[col]
            break
    else:
        df['suhu_aktual'] = df['prediksi']

# Konversi ke kelas: <=20 = dingin (0), >20 = hangat (1)
df['aktual_kelas'] = df['suhu_aktual'].apply(lambda x: 0 if x <= 20 else 1)
df['prediksi_kelas'] = df['prediksi'].apply(lambda x: 0 if x <= 20 else 1)

# Evaluasi
cm = confusion_matrix(df['aktual_kelas'], df['prediksi_kelas'])
tn, fp, fn, tp = cm.ravel()
akurasi = accuracy_score(df['aktual_kelas'], df['prediksi_kelas']) * 100
sensitivitas = tp / (tp + fn) * 100 if (tp + fn) != 0 else 0

print("\n=== EVALUASI METODE TSUKAMOTO ===")
print(f"Akurasi      : {akurasi:.2f}%")
print(f"Sensitivitas : {sensitivitas:.2f}%")

# Tampilkan seluruh hasil prediksi
print("\n=== HASIL PREDIKSI LENGKAP ===")
print(df[['suhu_luar', 'suhu_dalam', 'jumlah_orang', 'suhu_aktual', 'prediksi']].to_string(index=False))

# Simpan hasil prediksi ke file CSV
df.to_csv("hasil_prediksi_tsukamoto.csv", index=False)
print("\nHasil prediksi telah disimpan ke 'hasil_prediksi_sugeno.csv'")


=== EVALUASI METODE TSUKAMOTO ===
Akurasi      : 75.23%
Sensitivitas : 76.71%

=== HASIL PREDIKSI LENGKAP ===
 suhu_luar  suhu_dalam  jumlah_orang  suhu_aktual  prediksi
      22.0        23.5            10         19.5      19.2
      30.0        20.0            15         22.0      20.2
      17.0        19.5            25         21.0      19.6
      20.5        25.0            12         20.0      19.5
      34.5        20.0            30         17.0      16.6
      36.0        29.0            35         15.5      16.6
      32.0        22.5            18         22.0      22.2
      33.0        31.0            40         15.0      16.8
      28.0        23.0            27         21.5      23.2
      26.5        24.0            12         22.0      21.7
      21.5        28.0            16         20.0      20.5
      25.0        21.0            14         16.5      20.0
      17.0        26.5            35         17.0      23.5
      31.5        27.5            28         22.5